In [ ]:
import pickle
import datetime
import requests
import time as ti
import pandas as pd
import time
from itertools import count
from dateutil.parser import parse

In [ ]:
ditch_stop =['132','133','134','135','136','137','138', '139', '140', '141','142','174','175', '176']

def get_json_backend(date, page):
    url = 'http://192.168.99.100/api/trip/?agency=31&agency=45&agency=52&service=True&date=' + date
    results = []
    print("Retrieving backend trips for", date)
    first = True
    while True:

        r = requests.get(url)
        j = r.json()
        
        if first:
            print(j['count'])
            first = False
        
        #print(r.url)
        results = results + j['results']
        url = j['next']
        if not url:
            break
        
        page = page - 1
        if page == 0:
            break
            
    return results

def get_request_transport(params):
    base_url="http://transport.opendata.ch/v1/connections"
    r = requests.get(base_url, params=params)
    return r

def print_back(trip, f, t ):
    print(trip['start'], trip['count'], f, t, trip['headsign'])
    for stop in trip['stops']:
        print(stop['stop_id'], stop['stop_sequence'], stop['id'], stop['stop_name'])
        
def get_info_backend(trip):
    stop_dict = {stop['stop_id']: stop for stop in trip['stops'] if stop['stop_id'] not in ditch_stop}
    f = next((item['stop_id'] for item in trip['stops'] if item['stop_sequence'] == 0), None)
    t = next((item['stop_id'] for item in trip['stops'] if item['stop_sequence'] == trip['count']-1), None)    
    return stop_dict, f, t

def parse_time(t):
    try:
        return str(parse(t).time())
    except:
        return ""
    
def process_trip(trip, date, time):
    stop_backend, f, t = get_info_backend(trip)

    params = {
        'from': f,
        'to': t,
        'date': date,
        'time': time,
        'direct':1,
        'limit':1
    }
    
    trans_r = get_request_transport(params)
    
    try:
        sections_transport = get_info_transport(trans_r.json()['connections'])
    except Exception as e:
        print(trans_r.url)
        print(trans_r.json())
        raise ValueError("No connections") 
        
    for journey in sections_transport:
        # Verify we go through all stops
        #TODO should verify if start at the same time.
        if stop_backend.keys() == journey.keys(): 
            return trans_r.json()
    print()
    print("Error ##########")
    
    print(trans_r.url)
    print_back(trip, f, t)
    print("-  -  -")
    for journey in sections_transport:
        for k, stop in journey.items():
            print(stop['station']['id'], stop['station']['name'])
    
    raise ValueError("Stops mismatch") 
    
def buzzergen(period):
    nexttime = time.time() + period
    for i in count():
        now = time.time()
        tosleep = nexttime - now
        if tosleep > 0:
            time.sleep(tosleep)
            nexttime += period
        else:
            nexttime = now + period
        yield i, nexttime



def get_info_transport(connections): 
    passList = []
    for connection in connections:
        journey_stops = {}
        for section in connection['sections']:
            if section['journey']:
                journey_stops.update({stop['station']['id']: stop for stop in section['journey']['passList']})
                
        passList.append(journey_stops)
    
    return passList

------------

In [ ]:
buzzer = buzzergen(0.4)
directory = 'data/'
def get_capacity_data(date, page=1000, batch=1000):
    back_list = get_json_backend(date, page=page)
    print("Processing", len(back_list))
    
    day_dict = []
    error = []
    for i, trip in enumerate(back_list):  
        if i % batch == 0 and i != 0: 
            print("Saving *********", i)
            pickle.dump(day_dict, open(directory + 'occupancy_' + date + '.pkl', 'wb'))
        
        # Wait at least 0.5 seconds between each call to the transport API
        next(buzzer)
        
        try:
            transport_json = process_trip(trip, date, trip['start'])  
            
            out = {
                'date': date, 
                'backend': trip,
                'transport': transport_json,
            }
            
            day_dict.append(out)
            
        except ValueError as e:
            print("=>", e)
            error.append(trip)
        except Exception as e:
            print("=>",e)
            error.append(trip)
            
        ti.sleep(0.4)
    
    pickle.dump(day_dict, open(directory + 'occupancy_' + date + '.pkl', 'wb'))
    pickle.dump(error, open(directory + 'error_' + date + '.pkl', 'wb'))
    return error, day_dict
    

---------

In [ ]:
dates = ["2017-01-09", "2017-01-10", "2017-01-11", "2017-01-13", "2017-01-14", "2017-01-15", "2017-01-16"]

for date in dates:
    get_capacity_data(date, page=150)